<a href="https://colab.research.google.com/github/Titas23/IS-170-Machine-Learning/blob/6.-K-Nearest-Neighbors/HW7_Loan_Default.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder


In [ ]:
df = pd.read_csv('Loan_Default.csv')
df.head()

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
0,24890,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,...,EXP,758.0,CIB,25-34,to_inst,98.728814,south,direct,1.0,45.0
1,24891,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,...,EQUI,552.0,EXP,55-64,to_inst,NaN,North,direct,1.0,NaN
2,24892,2019,cf,Male,pre,type1,p1,l1,nopc,nob/c,...,EXP,834.0,CIB,35-44,to_inst,80.019685,south,direct,0.0,46.0
3,24893,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,...,EXP,587.0,CIB,45-54,not_inst,69.376900,North,direct,0.0,42.0
4,24894,2019,cf,Joint,pre,type1,p1,l1,nopc,nob/c,...,CRIF,602.0,EXP,25-34,not_inst,91.886544,North,direct,0.0,39.0


In [ ]:
# Take a look at the information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10976 entries, 0 to 10975
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         10976 non-null  int64  
 1   year                       10976 non-null  int64  
 2   loan_limit                 10722 non-null  object 
 3   Gender                     10976 non-null  object 
 4   approv_in_adv              10908 non-null  object 
 5   loan_type                  10975 non-null  object 
 6   loan_purpose               10968 non-null  object 
 7   Credit_Worthiness          10975 non-null  object 
 8   open_credit                10975 non-null  object 
 9   business_or_commercial     10975 non-null  object 
 10  loan_amount                10975 non-null  float64
 11  rate_of_interest           8291 non-null   float64
 12  Interest_rate_spread       8276 non-null   float64
 13  Upfront_charges            8063 non-null   flo

In [ ]:
# We see what the shape of the csv looks like
df.shape

(10976, 34)

In [ ]:
# We want to find out how many missing values there are and sum up the amount of missing values. 
df.isnull().sum()

ID                              0
year                            0
loan_limit                    254
Gender                          0
approv_in_adv                  68
loan_type                       1
loan_purpose                    8
Credit_Worthiness               1
open_credit                     1
business_or_commercial          1
loan_amount                     1
rate_of_interest             2685
Interest_rate_spread         2700
Upfront_charges              2913
term                            6
Neg_ammortization               9
interest_only                   1
lump_sum_payment                1
property_value               1152
construction_type               1
occupancy_type                  1
Secured_by                      1
total_units                     1
income                        657
credit_type                     1
Credit_Score                    1
co-applicant_credit_type        1
age                            16
submission_of_application      16
LTV           

In [ ]:
# We will drop any rows where we have missing values.
df=df.dropna(subset=['loan_limit','approv_in_adv','loan_purpose',
                     'rate_of_interest','Interest_rate_spread','Upfront_charges',
                     'term','Neg_ammortization','property_value','income',
                     'age','submission_of_application','LTV','dtir1'])
# You will notice that the shape has changed after we drop the missing values.
print(df.shape)

In [ ]:
# Now we just verify that we are not missing any values.
df.isnull().sum()

ID                           0
year                         0
loan_limit                   0
Gender                       0
approv_in_adv                0
loan_type                    0
loan_purpose                 0
Credit_Worthiness            0
open_credit                  0
business_or_commercial       0
loan_amount                  0
rate_of_interest             0
Interest_rate_spread         0
Upfront_charges              0
term                         0
Neg_ammortization            0
interest_only                0
lump_sum_payment             0
property_value               0
construction_type            0
occupancy_type               0
Secured_by                   0
total_units                  0
income                       0
credit_type                  0
Credit_Score                 0
co-applicant_credit_type     0
age                          0
submission_of_application    0
LTV                          0
Region                       0
Security_Type                0
Status  

In [ ]:
# We will transform any columns where the information is in string value over to digits. 
le = LabelEncoder()

label = le.fit_transform(df['loan_amount'])
label_1 = le.fit_transform(df['loan_type'])
label_2 = le.fit_transform(df['Credit_Worthiness'])
label_3 = le.fit_transform(df['submission_of_application'])
label_4 = le.fit_transform(df['Credit_Score'])
label_5 = le.fit_transform(df['approv_in_adv'])



df.drop('loan_amount', axis=1, inplace=True)
df.drop('loan_type', axis=1, inplace=True)
df.drop('Credit_Worthiness', axis=1, inplace=True)
df.drop('submission_of_application', axis=1, inplace = True)
df.drop('Credit_Score', axis=1, inplace = True)
df.drop('approv_in_adv', axis=1, inplace = True)



df['loan_amount']= label
df['loan_type']= label_1
df['Credit_Worthiness']= label_2
df['submission_of_application']= label_3
df['Credit_Score']= label_4
df['approv_in_adv']= label_5



In [ ]:
# our x value will measure the following columns
X = df[['loan_amount', 'loan_type', 'Credit_Worthiness','submission_of_application', 'Credit_Score','income','rate_of_interest','LTV']].values
X

array([[3.80000000e+01, 0.00000000e+00, 0.00000000e+00, ...,
        9.48000000e+03, 4.56000000e+00, 8.00196850e+01],
       [6.70000000e+01, 0.00000000e+00, 0.00000000e+00, ...,
        1.04400000e+04, 4.00000000e+00, 9.18865435e+01],
       [6.80000000e+01, 0.00000000e+00, 0.00000000e+00, ...,
        1.00800000e+04, 3.99000000e+00, 7.00892857e+01],
       ...,
       [2.70000000e+01, 0.00000000e+00, 0.00000000e+00, ...,
        8.16000000e+03, 3.75000000e+00, 7.64175258e+01],
       [4.40000000e+01, 0.00000000e+00, 0.00000000e+00, ...,
        1.34400000e+04, 4.50000000e+00, 5.50117925e+01],
       [1.00000000e+01, 1.00000000e+00, 0.00000000e+00, ...,
        3.90000000e+03, 3.99000000e+00, 8.54729730e+01]])

In [ ]:
# We will transform our values to a float decimal number.
X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))
X[0:5]

array([[ 0.40868446, -0.41074431, -0.19425717,  0.80413425,  1.15506026,
         0.32977319,  0.89556688,  0.49424216],
       [ 2.10080688, -0.41074431, -0.19425717, -1.24357345, -0.86106791,
         0.46822158, -0.14916671,  1.15589451],
       [ 2.15915593, -0.41074431, -0.19425717, -1.24357345,  1.41576649,
         0.41630343, -0.16782267, -0.05944037],
       [ 0.05859016, -0.41074431, -0.19425717,  0.80413425,  1.38100566,
        -0.3105506 ,  0.78363114,  0.44349855],
       [ 0.23363731, -0.41074431, -0.19425717,  0.80413425, -1.05225247,
        -0.23267339,  1.48322953,  0.42432403]])

In [ ]:
# Our Y value will measure if the candidates were approved in advance. 
y = df['approv_in_adv'].values
y [0:5]


array([1, 1, 1, 1, 0])

In [ ]:
# We will setup our training and testing for the model. 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.35)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)


Train set: (4719, 8) (4719,)
Test set: (2541, 8) (2541,)


In [ ]:
#Generating a model with K=5
#Import k-nearest neighbors classifier model
from sklearn.neighbors import KNeighborsClassifier

#Create KNN Classifier
knn=KNeighborsClassifier(n_neighbors=9)

#Train the model using the training sets
knn.fit(X_train,y_train)

#Predict the response for test dataset
y_pred=knn.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

#Model accuracy, how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))



Accuracy: 0.8421881149153877


From the result above, the classification rate is about 82%. Based on the results we have a chance that 82% of the candidates will not default on their home loan, however there is an 18% chance of the candidates that are at risk of defaulting on their home loan. 